# Exploring the Azrieli & Sons Data

## Libraries and data loading

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Looking for problems in the data

In [2]:
def calc_nan_df(df):
    nan_summary = df.isna().sum()
    nan_summary = nan_summary[nan_summary > 0].sort_values(ascending=False)
    nan_summary = nan_summary.reset_index()
    nan_summary.columns = ['Column', 'NaN Count']
    nan_summary['NaN Ratio'] = np.round(nan_summary['NaN Count'] / df.shape[0],2)
    return nan_summary

data_path = r'C:\Users\user\Desktop\CV Training Guy\CV_training\basic\python\Basic - Azrieli & Sons data'
old_drivers_df = pd.read_csv(os.path.join(data_path, 'Drivers_with_kviut.csv'))
new_drivers_df = pd.read_csv(os.path.join(data_path, 'new_drivers.csv'))
taarif_df = pd.read_csv(os.path.join(data_path, 'taarif.csv'))
trips_data_list = os.listdir(os.path.join(data_path, 'trips_data'))

### Drivers, old and new

In [3]:
new_drivers_df

,Unnamed: 0,birthdate,gender,id,vetek
0,0,19.06.1967,F,943,232.0
1,1,"february 13, 1979",M,966,28.0
2,2,19/11/1993,M,250,48.0
3,3,02/02/1961,NaN,156,272.0
4,4,07/03/1965,M,67,130.0
...,...,...,...,...,...
95,95,16/08/1990,M,798,144.0
96,96,05/06/1967,M,865,103.0
97,97,02/10/1972,M,399,312.0
98,98,16/03/1979,M,509,172.0


First column (id) is redundant.

Second column (date) has different formats, which should be unified.

Third column (male) as above, some missing values as well. Should we keep unknown as a category or remove it? ASK

Rest seems ok.

In [4]:
calc_nan_df(new_drivers_df)

,Column,NaN Count,NaN Ratio
0,gender,3,0.03


In [5]:
old_drivers_df['gender'].value_counts()

gender
M          674
male        63
boy         37
unknown     37
m           36
F           17
female      10
girl         6
none         4
woman        3
Name: count, dtype: int64

Note that the vetek in the new drivers column is measured probably in days while in the old drivers column it is measured in years, should be unified.

Similar problems with the columns mentioned above.

In [6]:
calc_nan_df(old_drivers_df)

,Column,NaN Count,NaN Ratio
0,gender,13,0.01
1,birthdate,10,0.01


In [7]:
old_ids = old_drivers_df['id'].unique()
new_ids = new_drivers_df['id'].unique()
# intersect
common_ids = np.intersect1d(old_ids, new_ids)
common_ids

array([], dtype=int64)

### Salary

In [8]:
taarif_df

,customer,basic_taarif,extra_milage,night_bonus,weekend_bonus
0,NaN,NaN,NaN,NaN,NaN
1,jerusalem_muni,8.0,40.0,50.0,30.0
2,intel,8.0,6.0,20.0,0.0
3,mizranei_kfar_saba,15.0,4.0,0.0,25.0
4,dbs,10.0,8.0,0.0,25.0
5,tnuva,6.0,3.0,80.0,0.0
6,angel,1.0,5.0,100.0,0.0
7,telecommunication_ltd,7.0,7.0,10.0,40.0
8,osem,12.0,4.0,20.0,0.0
9,iaf,20.0,10.0,200.0,500.0


First row is just NaNs.

**Electricity** - missing data in extra milage (really missing or meant 0.0?), abnormal weekend bonus. ASK

**Bituch Leumi** - all data is missing. ASK

### Trips

Now for the trips data, noticed there are some duplicate files, meaning files with the same name (the duplicate usually denoted with and appended '(2)' after the file name).

In [9]:
trips_data_duplicates_list = [trip.split('.')[0] for trip in trips_data_list if '(' in trip]
whats_inside_parentheses = np.unique([trip.split('(')[1].split(')')[0] for trip in trips_data_duplicates_list])

In [10]:
trips_data_duplicates_list

['april 2015_4781355(2)',
 'april 2015_4781424(2)',
 'april 2015_4781442(2)',
 'april 2015_4781451(2)',
 'april 2015_4781460(2)',
 'april 2015_4781495(2)',
 'april 2015_4781695(2)',
 'april 2015_4781849(2)',
 'august 2015_4781463(2)',
 'august 2015_4781591(2)',
 'august 2015_4781734(2)',
 'august 2015_4781737(2)',
 'august 2015_4781747(2)',
 'august 2015_4781800(2)',
 'december 2015_4781471(2)',
 'december 2015_4781525(2)',
 'december 2015_4781545(2)',
 'december 2015_4781631(2)',
 'december 2015_4781759(2)',
 'december 2015_4781798(2)',
 'february 2015_4781400(2)',
 'february 2015_4781409(2)',
 'february 2015_4781426(2)',
 'february 2015_4781469(2)',
 'february 2015_4781473(2)',
 'february 2015_4781520(2)',
 'february 2015_4781525(2)',
 'february 2015_4781551(2)',
 'february 2015_4781602(2)',
 'february 2015_4781654(2)',
 'february 2015_4781710(2)',
 'february 2015_4781723(2)',
 'february 2015_4781848(2)',
 'january 2015_4781374(2)',
 'january 2015_4781477(2)',
 'january 2015_4781505(

In [11]:
round(len(trips_data_duplicates_list) / len(trips_data_list), 2)

0.02

2 percent of the files are duplicates, can't ignore that. Let's see what is inside the parentheses, i.e. if the duplicate denoter is always '(2)'.

In [12]:
whats_inside_parentheses

array(['2'], dtype='<U1')

Thus duplicates are always denoted with a postfix of (2). Let see if there is a pattern to the duplicates, compared to the original files.

In [13]:
original_duplicate_tuples = [(x.split('(')[0] + '.csv', x + '.csv') for x in trips_data_duplicates_list]
# convert each path to dataframe and check if they are the same in each tuple
tuple_df = []
equal_array = []
for original, duplicate in original_duplicate_tuples:
    original_df = pd.read_csv(os.path.join(data_path, 'trips_data', original))
    duplicate_df = pd.read_csv(os.path.join(data_path, 'trips_data', duplicate))
    tuple_df.append((original_df, duplicate_df))
    # check if the dataframes are the same
    equal_array.append(original_df.equals(duplicate_df))

In [14]:
sum(equal_array)

111

In [15]:
sum(equal_array) / len(equal_array)

1.0

Thus the duplicates are entirely redundant, and we can remove them.

Let's inspect some individual files to look for more problems.

In [16]:
random_idx = np.random.randint(0, len(trips_data_list))
print(random_idx)

2961


In [17]:
shift = 0
random_trip_data_df = pd.read_csv(os.path.join(data_path, 'trips_data', trips_data_list[random_idx+shift]))
print(trips_data_list[random_idx+shift])
random_trip_data_df.sort_values('km', ascending=False).head(10)

july 2015_4781761.csv


,Unnamed: 0,customer,driver_id,end_time,km,start_time
1,2,yes,980,2015-07-02 08:05:00,559.955750,2015-07-02 01:21:00
13,14,osem,293,2015-07-08 08:11:00,521.560602,2015-07-08 01:20:00
4,5,intel,908,2015-07-03 21:13:00,509.510200,2015-07-03 14:37:00
12,13,yes,525,2015-07-07 18:46:00,454.947073,2015-07-07 12:08:00
2,3,yes,374,2015-07-02 17:39:00,433.071366,2015-07-02 12:19:00
10,11,osem,733,2015-07-06 19:50:00,431.797707,2015-07-06 14:34:00
0,1,electricity,257,2015-07-01 21:38:00,386.195159,2015-07-01 15:30:00
9,10,intel,214,2015-07-06 07:40:00,335.078748,2015-07-06 03:53:00
11,12,electricity,485,2015-07-07 06:26:00,255.153689,2015-07-07 02:21:00
14,15,aminach,143,2015-07-08 17:59:00,249.204678,2015-07-08 14:27:00


Is it (?) suspicious to see drivers drive 3000 km on the same job, during 2 days?

In [18]:
a = pd.unique(random_trip_data_df['customer'])
b = pd.unique(taarif_df['customer'])
print(a)
print(b)

['electricity' 'yes' 'hot' 'intel' 'osem' 'bituch_leumi' 'tara' 'aminach']
[nan 'jerusalem_muni' 'intel' 'mizranei_kfar_saba' 'dbs' 'tnuva' 'angel'
 'telecommunication_ltd' 'osem' 'iaf' 'tara' 'electricity' 'bituch_leumi']


In [19]:
# whats in a but not in b
a[~np.isin(a, b)]

array(['yes', 'hot', 'aminach'], dtype=object)

It seems the salary table doesn't have customers that appear on the trips table, which is a problem. ASK

Let's unify all the trip tables after while disregarding the duplicates.

In [20]:
trips_data_no_dups_list = [trip for trip in trips_data_list if '(2)' not in trip]
trips_dfs = [pd.read_csv(os.path.join(data_path, 'trips_data', trip)) for trip in trips_data_no_dups_list]
trips_df = pd.concat(trips_dfs, ignore_index=True)
trips_df.head()

,Unnamed: 0,customer,driver_id,end_time,km,start_time
0,0,iaf,570,2015-04-01 09:36:00,672.312716,2015-04-01 01:55:00
1,1,yes,822,2015-04-01 16:17:00,96.048221,2015-04-01 15:12:00
2,2,intel,437,2015-04-02 04:15:00,90.727997,2015-04-02 02:49:00
3,3,yes,70,2015-04-02 15:41:00,216.595350,2015-04-02 12:41:00
4,4,jerusalem_muni,446,NaN,96.437630,2015-04-03 00:15:00


In [21]:
trips_df

,Unnamed: 0,customer,driver_id,end_time,km,start_time
0,0,iaf,570,2015-04-01 09:36:00,672.312716,2015-04-01 01:55:00
1,1,yes,822,2015-04-01 16:17:00,96.048221,2015-04-01 15:12:00
2,2,intel,437,2015-04-02 04:15:00,90.727997,2015-04-02 02:49:00
3,3,yes,70,2015-04-02 15:41:00,216.595350,2015-04-02 12:41:00
4,4,jerusalem_muni,446,NaN,96.437630,2015-04-03 00:15:00
...,...,...,...,...,...,...
318176,49,iaf,549,2015-09-25 18:41:00,338.646624,2015-09-25 14:02:00
318177,50,yes,398,2015-09-26 08:07:00,690.632760,2015-09-26 00:11:00
318178,51,tnuva,924,2015-09-26 16:13:00,74.532361,2015-09-26 15:12:00
318179,52,tara,884,2015-09-27 06:21:00,534.599297,2015-09-27 00:05:00


In [22]:
a = pd.unique(trips_df['customer'])
b = pd.unique(taarif_df['customer'])
a[~np.isin(a, b)]

array(['yes', 'aminach', 'hot'], dtype=object)

In [23]:
b[~np.isin(b, a)]

array([nan, 'mizranei_kfar_saba', 'dbs', 'telecommunication_ltd'],
      dtype=object)

Interesting, common sense can match those (see summary document).

## Trying to assemble the summary table

In [24]:
print(new_drivers_df.columns)
print(old_drivers_df.columns)
print(taarif_df.columns)
print(trips_df.columns)

Index(['Unnamed: 0', 'birthdate', 'gender', 'id', 'vetek'], dtype='object')
Index(['Unnamed: 0', 'birthdate', 'gender', 'id', 'vetek'], dtype='object')
Index(['customer', 'basic_taarif', 'extra_milage', 'night_bonus',
       'weekend_bonus'],
      dtype='object')
Index(['Unnamed: 0', 'customer', 'driver_id', 'end_time', 'km', 'start_time'], dtype='object')


In [25]:
new_drivers_df['vetek'] = new_drivers_df['vetek'].apply(lambda x: x / 365)
new_drivers_df.drop('Unnamed: 0', axis=1, inplace=True)
old_drivers_df.drop('Unnamed: 0', axis=1, inplace=True)
trips_df.drop('Unnamed: 0', axis=1, inplace=True)

In [26]:
taarif_df['customer']

0                       NaN
1            jerusalem_muni
2                     intel
3        mizranei_kfar_saba
4                       dbs
5                     tnuva
6                     angel
7     telecommunication_ltd
8                      osem
9                       iaf
10                     tara
11              electricity
12             bituch_leumi
Name: customer, dtype: object

In [27]:
taarif_df.replace({
    'telecommunication_ltd': 'hot',
    'dbs': 'yes',
    'mizranei_kfar_saba': 'aminach'
}, inplace=True)
taarif_df.drop(axis=0, index=0, inplace=True)

In [28]:
taarif_df

,customer,basic_taarif,extra_milage,night_bonus,weekend_bonus
1,jerusalem_muni,8.0,40.0,50.0,30.0
2,intel,8.0,6.0,20.0,0.0
3,aminach,15.0,4.0,0.0,25.0
4,yes,10.0,8.0,0.0,25.0
5,tnuva,6.0,3.0,80.0,0.0
6,angel,1.0,5.0,100.0,0.0
7,hot,7.0,7.0,10.0,40.0
8,osem,12.0,4.0,20.0,0.0
9,iaf,20.0,10.0,200.0,500.0
10,tara,6.0,6.0,50.0,30.0


In [29]:
trips_df.head()

,customer,driver_id,end_time,km,start_time
0,iaf,570,2015-04-01 09:36:00,672.312716,2015-04-01 01:55:00
1,yes,822,2015-04-01 16:17:00,96.048221,2015-04-01 15:12:00
2,intel,437,2015-04-02 04:15:00,90.727997,2015-04-02 02:49:00
3,yes,70,2015-04-02 15:41:00,216.595350,2015-04-02 12:41:00
4,jerusalem_muni,446,NaN,96.437630,2015-04-03 00:15:00


In [30]:
calc_nan_df(trips_df)

,Column,NaN Count,NaN Ratio
0,start_time,3149,0.01
1,end_time,3083,0.01


In [31]:
calc_nan_df(trips_df[trips_df['start_time'].isna()])

,Column,NaN Count,NaN Ratio
0,start_time,3149,1.00
1,end_time,43,0.01


In [32]:
calc_nan_df(trips_df[trips_df['end_time'].isna()])

,Column,NaN Count,NaN Ratio
0,end_time,3083,1.00
1,start_time,43,0.01


In [40]:
trips_df['end_time'] = trips_df['end_time'].astype("datetime64[ns]")
trips_df['start_time'] = trips_df['start_time'].astype("datetime64[ns]")

In [45]:
a = trips_df['end_time'][0] - trips_df['start_time'][0]
a.total_seconds() / 3600

7.683333333333334

In [53]:
old_drivers_df.iloc[[89,106,108], :]

,birthdate,gender,id,vetek
89,01/01/1900,M,572,7.6
106,NaN,M,779,18.0
108,25/04/1970,male,821,4.4


In [50]:
old_drivers_df[old_drivers_df['birthdate'].isna()]

,birthdate,gender,id,vetek
106,NaN,M,779,18.0
238,NaN,M,883,15.3
315,NaN,M,388,1.6
425,NaN,M,547,7.5
490,NaN,m,397,12.9
570,NaN,M,785,14.7
601,NaN,M,871,7.2
608,NaN,M,347,15.2
655,NaN,M,668,14.9
801,NaN,M,690,8.8


In [60]:
taarif_df.iloc[:2]

,customer,basic_taarif,extra_milage,night_bonus,weekend_bonus
1,jerusalem_muni,8.0,40.0,50.0,30.0
2,intel,8.0,6.0,20.0,0.0


In [69]:
trips_df.iloc[[175, 174, 4, 12509], :]

,customer,driver_id,end_time,km,start_time
175,tnuva,197,2015-04-07 21:37:00,524.353643,2015-04-07 14:40:00
174,tnuva,984,2015-04-06 11:01:00,524.839597,NaT
4,jerusalem_muni,446,NaT,96.437630,2015-04-03 00:15:00
12509,yes,653,NaT,454.815943,NaT


In [67]:
trips_df[trips_df['start_time'].isna()][trips_df['end_time'].isna()]

C:\Users\user\AppData\Local\Temp\ipykernel_12756\2428092740.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  trips_df[trips_df['start_time'].isna()][trips_df['end_time'].isna()]


,customer,driver_id,end_time,km,start_time
12509,yes,653,NaT,454.815943,NaT
22331,yes,643,NaT,57.974586,NaT
23663,osem,705,NaT,197.556857,NaT
24202,intel,52,NaT,99.175149,NaT
32119,tnuva,295,NaT,168.653256,NaT
45825,tnuva,414,NaT,630.036986,NaT
47956,jerusalem_muni,201,NaT,122.775826,NaT
56289,bituch_leumi,136,NaT,285.179849,NaT
81486,iaf,775,NaT,196.836166,NaT
90347,hot,620,NaT,91.182702,NaT
